In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import shape
import rasterio 
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio.mask
from pyproj import CRS
import os
os.environ['USE_PYGEOS'] = '0'

from util.files import *
from util.const import *
from util.ddfs import *

In [3]:
# FIPS will be passed in as an argument, one day...
FIPS = '34007'
# STATE ABBR and NATION will be derived from FIPS, one day...
STATEABBR = 'NJ'
NATION = 'US'

# Process - everything ends up at county level and clipped

## Process clip data

In [4]:
# For our case study, we are going to focus on Gloucester City, NJ
# Our config.yaml loads in a county indexed clip file
# so that we can restrict all our data to the GC boundaries

In [5]:
# Read in the data we downloaded from the county's REST API server
clip_filep = join(REF_DIR_R, FIPS, 'clip.json')
with open(clip_filep) as f:
    clip_data = json.load(f)

In [6]:
# Use pandas to get the data in a form that is easier
# to turn into a geodataframe for clipping
clip_df = pd.json_normalize(clip_data['features'])
# We want to make a polygon out of the geometry coordinates
# We can access that from the original json object
clip_geo = [shape(i['geometry']) for i in clip_data['features']]
# We can create a geodataframe of clip_df by adding clip_geo
# as its geometry column
clip_gdf = gpd.GeoDataFrame(clip_df,
                            crs=CLIP_CRS,
                            geometry=clip_geo)

# We can clean up the gdf by removing the
# type, id, geometry.type and geometry.coordinates columns
drop_col = ['type', 'id', 'geometry.type', 'geometry.coordinates']
clip_gdf = clip_gdf.drop(columns=drop_col)

# Write the file out to interim
clip_out_filep = join(FI, 'ref', FIPS, 'clip.gpkg')
prepare_saving(clip_out_filep)
clip_gdf.to_file(clip_out_filep,
                 driver='GPKG')

## Process NSI

In [4]:
# The NSI comes with all the data necessary for performing a standard 
# flood risk assessment. It is still useful to process the raw data.
# Here, we subset to residential properties with 1 to 2 stories
# and save as a geodataframe. These are the types of residences we have
# multiple depth-damage functions for and a literature base to draw 
# from to introduce uncertainty in these loss estimates

In [7]:
# Read NSI, reset index upon reading
# TODO - this will be json...
nsi_filep = join(EXP_DIR_R, FIPS, 'nsi.pqt')
nsi_full = pd.read_parquet(nsi_filep).reset_index(drop=True)

In [8]:
# Convert to gdf
# This is useful for some spatial joins we need to perform
# Convert to geodataframe
geometry = gpd.points_from_xy(nsi_full['properties.x'],
                             nsi_full['properties.y'])
# The NSI CRS is EPSG 4326
nsi_gdf = gpd.GeoDataFrame(nsi_full, geometry=geometry,
                           crs=NSI_CRS)

# Drop the following columns
drop_cols = ['type', 'geometry.type', 'geometry.coordinates']
nsi_gdf = nsi_gdf.drop(columns=drop_cols)

# Remove "properties" from columns
col_updates = [x.replace("properties.", "") for x in nsi_gdf.columns]
nsi_gdf.columns = col_updates

In [9]:
# Subset to residential properties and update
# RES 1 - single family
# RES 2 - manufactured home
# RES 3 - multifamily (but could fit into a depth-damage function
# archetype depending on # stories)
# We are going to use RES1 for this case-study
# It is the only occtype with hazus and naccs
# DDFs and has less ambiguous classification

# occtype category for easier use in loss estimation steps

# Get residential structures
nsi_res = nsi_gdf.loc[nsi_gdf['occtype'].str[:4] == 'RES1']

# For this case-study, don't use any building with more 
# than 2 stories
res1_3s_ind = nsi_res['num_story'] > 2
# Final residential dataframe
res_f = nsi_res.loc[~res1_3s_ind]

In [10]:
# Subset to relevant columns
cols = ['fd_id', 'occtype', 'found_type', 'cbfips',
        'ftprntsrc', 'found_ht', 'val_struct',
        'val_cont', 'source', 'firmzone', 'ground_elv_m',
        'geometry']

res_out = res_f.loc[:,cols]

# Clip to our clip boundary
# They are in the same CRS
nsi_clip_out = gpd.clip(res_out, clip_gdf)

In [16]:
# Write out to interim/exposure/FIPS/
# Single family homes -- sf
EXP_OUT_FILEP = join(EXP_DIR_I, FIPS, 'nsi_sf.gpkg')
prepare_saving(EXP_OUT_FILEP)
nsi_clip_out.to_file(EXP_OUT_FILEP, driver='GPKG')

## Process Depth-Damage Functions

In [19]:
# Generally, we will process these DDFs the same way since they
# are written in mostly the same format
# However, there are a few preprocessing steps necessary for the hazus
# ddfs. Also, there are some differences for NACCS vs. HAZUS
# shallow uncertainty representation

In [51]:
# Read depth damage functions
ddf_filedir = join(VULN_DIR_UZ, "physical", NATION)
naccs = pd.read_csv(join(ddf_filedir, "naccs_ddfs.csv"))
hazus = pd.read_csv(join(ddf_filedir, "haz_fl_dept.csv"))

### HAZUS DDFs

In [52]:
# First, preprocessing for hazus ddfs
# For basements, use FIA (MOD.) which does one and two floors by
# A and V zones
# For no basements, use USACE - IWR
# which does one and two floor, no flood zone specified
# 106: FIA (MOD.) 1S WB A zone
# 114: "" V zone
# 108: FIA (MOD.) 1S WB A zone
# 116: "" V zone
# 129: USACE - IWR 1S NB
# 130: USCAE - IWR 2S+ NB
# For elevating homes, we can use Pile foundation DDFs
# from USACE - Wilmington
# 178 - 1S Pile Foundation
# 183 - 2S Pile Foundation
# These are no basement homes, so to speak
# The USACE New Orleans DDFs have some pier foundation
# DDFs with fresh & salt water and long & short duration
# but this does not appear to apply to out study area
# Subset to DmgFnId in the codes above
dmg_ids = [106, 108, 114, 116, 129, 130, 178, 183]
hazus_res = hazus[(hazus['DmgFnId'].isin(dmg_ids)) & 
                  (hazus['Occupancy'] == 'RES1')]

# Make occtype column in the same form that the NSI has
# e.g. RES1-1SNB
# Add column for A or V zone
# Note: outside SFHA basement homes will take A zone
# What other option do we have? 

# Split Description by comma. 
# The split[0] element tells us stories (but description sometimes
# says floors instead of story...)
# Can get around this issue by looking at first word
# The split[1] element
# tells us w/ basement or no basement. Use this to create occtype
desc = hazus_res['Description'].str.split(',')
s_type = desc.str[0].str.split(' ').str[0]
s_type = s_type.str.replace('one', '1').str.replace('two', '2')
b_type = desc.str[1].str.strip()
# Below, we are just trying to get archetypes like
# 1SNB, 2SWB, 1SPL -- for pile foundation
occtype = np.where(b_type == 'w/ basement',
                   s_type + 'SWB',
                   s_type + 'SNB')
occtype = np.where(b_type == 'Pile foundation',
                   s_type + 'SPL',
                   occtype)
# Some of these HAZUS DDFs require us to keep track of the
# flood zone they're in
# I don't think this matters for our case study since
# there are no high wave coastsal zones
# This line is designed to work specifically 
# with the way the descriptions
# are written out for the DDFs used in this case study
fz = desc.str[-1].str.lower().str.replace('structure', '').str.strip()

In [53]:
# Need occtype, flood zone, depth_ft, and rel_dam columns
# Follow steps from naccs processing to get depth_ft and rel_dam
# First, drop unecessary columns
# Don't need Source_Table, Occupy_Class, Cover_Class, empty columns
# Description, Source, DmgFnId, Occupancy and first col (Unnamed: 0)
# because index was written out
# Don't need all na columns either (just for automobiles, apparently)
hazus_res = hazus_res.loc[:,[col for col in hazus_res.columns if 'ft' in col]]
hazus_res = hazus_res.dropna(axis=1, how='all')
# Add the occtype and fld_zone columns
hazus_res = hazus_res.assign(occtype=occtype,
                             fld_zone=fz.str[0])

# Then, occtype and fld_zone as index and melt rest of columns. 
idvars = ['occtype', 'fld_zone']

# Get a tidy ddf back
hazus_melt = tidy_ddfs(hazus_res, idvars)

# Delete depth_str and pctdam and standardize
# column names
# Since we just have the building types, call this
# bld_type instead of occtype
dropcols = ['depth_str', 'pct_dam', 'occtype', 'fld_zone']

# We create an "id" col for the ddfs
# Our key for HAZUS is bld_type & fld_zone
ddf_id = np.where(hazus_melt['fld_zone'].notnull(),
                  hazus_melt['occtype'] + '_' + hazus_melt['fld_zone'],
                  hazus_melt['occtype'])

# Add this to our dataframe so that we can drop bld_type & fld_zone
# Easier to have the flood zone as a capital letter
# It's lower case because of earlier code to do
# some processing
hazus_melt = hazus_melt.assign(ddf_id=pd.Series(ddf_id).str.upper())
# Drop columns
hazus = hazus_melt.drop(columns=dropcols)

In [54]:
# Now we're going to process this tidy dataframe into a dictionary
# for easier ensemble generation

# After we get this new column, we are going to create two
# new columns based on the +/- .3*pt_estimate (30% uncertainty) 
# assumption from Maggie's paper 
# (https://www.nature.com/articles/s41467-020-19188-9)
# We will take the ddf_id, depth_ft, and these two columns
# to do the same thing as before for the dict of dicts
# We need to use max(0, ) and min(1, ) to make sure the +/- .3
# doesn't lead to negative losses, greater than 100% losses
# Since Maggie's paper, though, there have been studies
# suggesting that the damage distribution at each depth
# follows more of a long upper tailed Beta distribution. 
# While we don't have parameters for this, we can at least 
# represent a wider upper tail. So, -.3 and +.5 can better
# represent this
# A key assumption is that
# we can round depths to the nearest value in the
# dictionary to estimate their loss. There is no guidance in the
# use of DDFs about interpolating between values given on the DDF
# NFIP assessed damages data (recently released with the new v2 of
# the NFIP claims) only provides depth in feet, rounded to the
# nearest foot. So, any uncertainty surrounding the depth-damage
# relationship for any foot should include some component of 
# measurement error in representing some non rounded depth value
# to the rounded value and estimating a relationship
# To implement this, we will round all depths to the nearest foot
# before we check for whether they are inside the bounds for
# estimating losses with a particular depth-damage function
# Because of this, rounding the parameters to the nearest
# hundredth is a much lower order concern
dam_low = np.maximum(0, hazus['rel_dam'] - .3*hazus['rel_dam']).round(2)
dam_high = np.minimum(1, hazus['rel_dam'] + .5*hazus['rel_dam']).round(2)

# Add these columns into our dataframe
hazus = hazus.assign(low=dam_low,
                     high=dam_high)

# Next, we are getting our params
# Since we're uniformly sampling from the range
# our params are endpoints
# We don't need the rel_dam column anymore 
hazus_f = hazus.drop(columns='rel_dam')

# We are going to write out hazus_f 
# In generating the ensemble for losses
# we are going to merge this dataframe
# with our structure ensemble - merged with
# depths. So, on haz_depth & depth_ft from hazus_f
# plus the structure archetype, we can get
# the rel_dam parameters. We will draw from this
# and get the rel_dam realization for this
# state of the world
# But, he way this data is stored requires a few assumptions
# about loss estimation
# First, any depths below that lowest depth have 0 loss
# Second, any depths above the highest depth have the same
# loss as the highest depth 
# To implement this, we will check depths (after drawing from their
# distribution at each location) for whether they are inside
# the range of the particular DDF which can be defined with 
# conastants. If below, loss is 0. If above, swap with
# the upper bound
# We are going to do the processing for that in the script
# where we generate the loss ensemble using a helper function


### NACCS DDFs

In [ ]:
# For NACCS, we have the RES 1 DDFs
# For elevation, we have RES-OPEN and RES-ENC
# These are very similar in terms of damages so
# only need to retain RES-OPEN for simplicity for
# our current case study

In [93]:
# NACCS need some preprocessing as well
# First, subset to the relevant Occupancy types
# We want to end up with ddf ids 1swb, open, etc.
# don't need to keep the RES1- part for this case study
naccs['res_type'] = naccs['Occupancy'].str.split('-').str[0]
naccs['bld_type'] = naccs['Occupancy'].str.split('-').str[1]
occ_types = ['1SWB', '2SWB', '1SNB', '2SNB', 'OPEN']
naccs_res = naccs.loc[(naccs['bld_type'].isin(occ_types)) &
                      ((naccs['res_type'] == 'RES1') |
                       (naccs['res_type'] == 'RES'))]

# Next, drop columns we don't need
drop_cols = ['Description', 'Source', 'Occupancy', 'res_type']
naccs_res = naccs_res.drop(columns=drop_cols)

# Rename DamageCategory
naccs_res = naccs_res.rename(columns={'DamageCategory': 'dam_cat'})

In [99]:
# Now get the melted dataframe
idvars = ['bld_type', 'dam_cat']
naccs_melt = tidy_ddfs(naccs_res, idvars)

# Drop columns we don't need
drop_cols = ['depth_str', 'pct_dam']
naccs_f = naccs_melt.drop(columns=drop_cols)

# We want to pivot the dataframe so that Min/ML/Max are our columns
naccs_piv = naccs_f.pivot(index=['bld_type', 'depth_ft'],
                          columns='dam_cat')['rel_dam'].reset_index()

# These our the parameters for the triangular distributions
# we will draw from
# Rename them as the parameter names - left, mode, right
naccs_out = naccs_piv.rename(columns={'Min': 'left',
                                      'ML': 'mode',
                                      'Max': 'right'})


### Save our processed ddfs

In [100]:
ddf_out_dir = join(VULN_DIR_I, 'physical')
prepare_saving(ddf_out_dir)

# Save hazus_f as hazus_ddfs.csv
hazus_f.to_csv(join(VULN_DIR_I, 'hazus_ddfs.csv'),
               index=False)

# Save naccs_out as naccs_ddfs.csv
naccs_out.to_csv(join(VULN_DIR_I, 'naccs_ddfs.csv'),
                 index=False)

## Process Reference Data

In [ ]:
# glob for shp files in REF_DIR_UZ - we need to get these 
# clipped

# Process Social Vulnerability Data

In [ ]:
# need to process these all at the county level
# sovi is in VULN_DIR_UZ/social/{STATEABBR}/SoVI2010_{STATEABBR}/...
# can glob for .shp (no .shp.xml)

# Link - everything to structures

## Link NSI with Flood Zones

## Link NSI with Reference Data

## Link NSI with Social Vulnerability Data